In [47]:
import sys
sys.path.append("..")

import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.inspection import partial_dependence
from IncrementsRegression import model

INCREMENTS_PATH = "C:/SuryaMain/Python Projects/ChinookUL/ChinookSalmonResearchProject/data"

In [21]:
spring_age_paths = []
for file in os.listdir(INCREMENTS_PATH):
    if "spring_age" in file:
        spring_age_paths.append(os.path.join(INCREMENTS_PATH, file))

spring_age_data = pd.concat([pd.read_csv(file_path) for file_path in spring_age_paths], axis=0)

In [22]:
X_full = spring_age_data.loc[:, spring_age_data.columns != 'length']
y_full = spring_age_data.length
X_stock = X_full['stock']
X_full = X_full.drop('stock', axis=1)
X_full = pd.concat([X_full, X_stock], axis=1)


In [23]:
X_full = pd.get_dummies(X_full, columns=['stock'])
X_full = X_full.drop('Unnamed: 0', axis=1)

In [81]:
X_train, X_test, y_train, y_test = \
    train_test_split(X_full, y_full, test_size=0.1, random_state=45, shuffle=True)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [85]:
scaler = StandardScaler()
scaled_df_train = pd.DataFrame(scaler.fit_transform(X_train[['year']]))
scaled_df_train.rename(columns = {0:'year'}, inplace=True)
#scaler.transform(X_test[['year']])
X_train = X_train.loc[:, X_train.columns != 'year'].join(scaled_df_train)

scaled_df_test = pd.DataFrame(scaler.fit_transform(X_test[['year']]))
scaled_df_test.rename(columns = {0:'year'}, inplace=True)
X_test = X_test.loc[:, X_test.columns != 'year'].join(scaled_df_test)

,SSTarc_summer,SSTarc_winter,PDO,WA_Pinks,ALPI,Pinks,NPGO,MEI_summer,MEI_winter,BI,stock_one_YO_summer COWL 3,stock_one_YO_summer GREL 3,stock_one_YO_summer SAND 3,stock_one_YO_summer WILL 3,stock_one_YO_summer WIND 3,stock_one_YO_summer YOCL 3,year
0,-0.088413,0.028948,0.368414,1.561447,-0.586785,-0.789755,-0.393901,-0.276667,-0.476667,1.613275,1,0,0,0,0,0,-1.074679
1,0.786593,0.509610,1.123033,-0.799018,0.414025,-0.180107,-0.972837,-0.540000,-0.760000,-1.131231,0,1,0,0,0,0,-0.053773
2,0.909630,-0.415323,-1.000050,1.814308,0.021707,2.699641,0.343829,-0.576667,-0.086667,0.679577,0,0,1,0,0,0,1.523991
3,0.317672,0.370556,-0.211914,-0.799018,1.903230,0.204636,0.544553,-1.490000,2.310000,-0.367297,0,0,0,0,0,1,0.131846
4,-0.181002,0.458551,1.643988,0.193823,1.518919,-0.471888,0.429175,1.543333,1.176667,-0.763411,0,1,0,0,0,0,-0.889060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,1.676437,0.042689,0.080166,-0.799018,1.202663,-0.082339,0.209460,0.570000,-0.106667,0.141993,0,1,0,0,0,0,0.688704
588,1.676437,0.042689,0.080166,-0.799018,1.202663,-0.082339,0.209460,0.570000,-0.106667,0.141993,0,1,0,0,0,0,0.688704
589,-0.209636,0.045696,-0.672538,1.802508,0.249892,0.281872,-0.498915,0.640000,0.150000,0.255169,0,0,0,0,1,0,-0.517821
590,-1.036231,-0.497541,-1.654117,-0.799018,0.177834,-0.156216,1.476881,-0.026667,-0.780000,-0.819999,1,0,0,0,0,0,1.431181


In [86]:
params = {
    "n_estimators": 100,
    "oob_score": True,
    "min_impurity_decrease": 0.1
}
test_rf = model.RFRegressor(params=params)
test_rf.fit(X_train, y_train)

In [87]:
test_rf.rsquared

0.10397633233499926

In [1]:
print("Testing Update")

Testing Update
